In [1]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import scipy 
import numpy as np
import random as rd
import nltk
import pickle
import sklearn
from sklearn.neighbors import NearestNeighbors
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, ConfusionMatrixDisplay, confusion_matrix

%matplotlib inline
nltk.download('stopwords')

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\igna-\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Puntos Extra

## Red Neuronal con Keras

In [2]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
identity = pd.read_csv('train_identity.csv')
transaction = pd.read_csv('train_transaction.csv')

In [4]:
mergeado = identity.merge(transaction, left_on='TransactionID', right_on='TransactionID',
          how = 'right')
mergeado.drop(['TransactionID'], axis=1, inplace= True)

Spliteo el 20% con el delta tiempo mayor

In [5]:
test_lenght = int(144233*0.2)
mergeado.sort_values(by = "TransactionDT")
train_lenght = (144233- test_lenght)
train = mergeado.iloc[0:train_lenght ,:]
test = mergeado.iloc[(144233- test_lenght):144233,:]

Encodeo el train set y el validation set

Mean encoding en id_30 y OHE en las demas categoricas

In [6]:
train["id_30"].fillna("Desconocido",inplace =True)
mean_encoder_id_30 = train.groupby(['id_30'])['isFraud'].mean().to_dict()

In [7]:
train["id_31"].fillna("Desconocido",inplace =True)
vec = CountVectorizer().fit(train["id_31"])
bag_of_words = vec.transform(train["id_31"])
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
filtro31 = list(filter(lambda c: c[0].isalpha() and c[1] > 2000, words_freq))

In [8]:
train["DeviceInfo"].fillna("Desconocido",inplace =True)
vec = CountVectorizer().fit(train["DeviceInfo"])
bag_of_words = vec.transform(train["DeviceInfo"])
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
filtroDI = list(filter(lambda c: c[0].isalpha() and c[1] > 2000, words_freq))


In [9]:
def selector(x, vec):
  for item in vec:
    if x.find(item[0]):
      return item[0]
    else:
      return 'other'

In [10]:
train['id_30'] =  train['id_30'].map(mean_encoder_id_30)
train['id_31'] = train['id_31'].map(lambda x: selector(x, filtro31))
train['DeviceInfo'] = train['DeviceInfo'].map(lambda x: selector(x, filtroDI))

X_cat = ['ProductCD','card4','id_34','id_15','id_23','id_31','DeviceType','DeviceInfo' ,'card6','P_emaildomain','R_emaildomain','M1','M2','M3','M4','M5','M6','M7','M8','M9']

cont_df = train.drop(X_cat,axis =1)
cont_df['id_12'] = cont_df['id_12'].replace('Found',1).replace('NotFound',0).astype(float)
cont_df['id_13'] = cont_df['id_13'].astype(float).fillna(cont_df['id_13'].mean()) 
cont_df['id_16'] = cont_df['id_16'].replace('Found',1).replace('NotFound',0).astype(float)
cont_df['id_27'] = cont_df['id_27'].replace('Found',1).replace('NotFound',0).astype(float)
cont_df['id_28'] = cont_df['id_28'].replace('Found',1).replace('New',0).astype(float)
cont_df['id_29'] = cont_df['id_29'].replace('Found',1).replace('NotFound',0).astype(float)
cont_df['id_13'] = cont_df['id_13'].astype(float).fillna(cont_df['id_13'].mean()) 
cont_df['id_33'] = cont_df['id_33'].astype(str).map(lambda x : 'nan' if x.find('nan') != -1 else
                                               int(x.split('x')[0])*int(x.split('x')[1])).astype(float)
media = cont_df['id_33'].mean()
cont_df['id_33'] = cont_df['id_33'] /media
cont_df['id_35'] = cont_df['id_35'].replace('T',1).replace('F',0).astype(float)
cont_df['id_36'] = cont_df['id_36'].replace('T',1).replace('F',0).astype(float)
cont_df['id_37'] = cont_df['id_37'].replace('T',1).replace('F',0).astype(float)
cont_df['id_38'] = cont_df['id_38'].replace('T',1).replace('F',0).astype(float)

cont_df.apply(lambda x: x.fillna(x.mean(),inplace = True)) 
cont_df.apply(lambda x: x.fillna(0,inplace = True)) 
cat_df = train[X_cat].fillna('Others')

In [11]:
encoders = []
for columna in X_cat:    
    ohe = OneHotEncoder(handle_unknown = 'ignore')  # drop='first'
    encoded_data = (ohe.fit_transform(cat_df[[columna]].astype(str)).todense().astype(int))
    encoders.append(ohe)
    encoded_data = pd.DataFrame(encoded_data, index = cont_df.index ).add_prefix(columna[0:8])
    cont_df = pd.concat([cont_df, encoded_data], axis=1)
train = cont_df

In [12]:
test["id_30"].fillna("Desconocido",inplace =True)
test["id_31"].fillna("Desconocido",inplace =True)
test["DeviceInfo"].fillna("Desconocido",inplace =True)
                          
test['id_30'] =  test['id_30'].map(mean_encoder_id_30)
test['id_31'] = test['id_31'].map(lambda x: selector(x, filtro31))
test['DeviceInfo'] = test['DeviceInfo'].map(lambda x: selector(x, filtroDI))

X_cat = ['ProductCD','card4','id_34','id_15','id_23','id_31','DeviceType','DeviceInfo' ,'card6','P_emaildomain','R_emaildomain','M1','M2','M3','M4','M5','M6','M7','M8','M9']

cont_df = test.drop(X_cat,axis =1)
cont_df['id_12'] = cont_df['id_12'].replace('Found',1).replace('NotFound',0).astype(float)
cont_df['id_13'] = cont_df['id_13'].astype(float).fillna(cont_df['id_13'].mean()) 
cont_df['id_16'] = cont_df['id_16'].replace('Found',1).replace('NotFound',0).astype(float)
cont_df['id_27'] = cont_df['id_27'].replace('Found',1).replace('NotFound',0).astype(float)
cont_df['id_28'] = cont_df['id_28'].replace('Found',1).replace('New',0).astype(float)
cont_df['id_29'] = cont_df['id_29'].replace('Found',1).replace('NotFound',0).astype(float)
cont_df['id_13'] = cont_df['id_13'].astype(float).fillna(cont_df['id_13'].mean()) 
cont_df['id_33'] = cont_df['id_33'].astype(str).map(lambda x : 'nan' if x.find('nan') != -1 else
                                               int(x.split('x')[0])*int(x.split('x')[1])).astype(float)
media = cont_df['id_33'].mean()
cont_df['id_33'] = cont_df['id_33'] /media
cont_df['id_35'] = cont_df['id_35'].replace('T',1).replace('F',0).astype(float)
cont_df['id_36'] = cont_df['id_36'].replace('T',1).replace('F',0).astype(float)
cont_df['id_37'] = cont_df['id_37'].replace('T',1).replace('F',0).astype(float)
cont_df['id_38'] = cont_df['id_38'].replace('T',1).replace('F',0).astype(float)

cont_df.apply(lambda x: x.fillna(x.mean(),inplace = True)) 
cont_df.apply(lambda x: x.fillna(0,inplace = True)) 
cat_df = test[X_cat].fillna('Others')

In [13]:
cont = 0 
for columna in X_cat:  
    encoded_data = (encoders[cont].transform(cat_df[[columna]].astype(str)).todense().astype(int))
    encoded_data = pd.DataFrame(encoded_data, index = cont_df.index ).add_prefix(columna[0:8])
    cont_df = pd.concat([cont_df, encoded_data], axis=1)
    cont += 1
test = cont_df


In [14]:
X_train = train.drop(['isFraud'], axis=1)
y_train = train.loc[:, 'isFraud']

In [15]:
X_test= test.drop(['isFraud'], axis=1)
y_test = test.loc[:, 'isFraud']

In [32]:
X_train

,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,...,M62,M70,M71,M72,M80,M81,M82,M90,M91,M92
0,-8.21,"136,512.62",0.08,-0.06,1.94,-5.92,14.28,-39.95,0.13,-0.34,...,1,0,1,0,0,1,0,0,1,0
1,-8.21,"136,512.62",0.08,-0.06,1.94,-5.92,14.28,-39.95,0.13,-0.34,...,1,0,1,0,0,1,0,0,1,0
2,-8.21,"136,512.62",0.08,-0.06,1.94,-5.92,14.28,-39.95,0.13,-0.34,...,0,1,0,0,1,0,0,1,0,0
3,-8.21,"136,512.62",0.08,-0.06,1.94,-5.92,14.28,-39.95,0.13,-0.34,...,0,0,1,0,0,1,0,0,1,0
4,0.00,"70,787.00",0.08,-0.06,1.94,-5.92,14.28,-39.95,0.13,-0.34,...,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115382,-5.00,"82,511.00",0.08,-0.06,0.00,0.00,14.28,-39.95,0.13,-0.34,...,0,0,1,0,0,1,0,0,1,0
115383,-8.21,"136,512.62",0.08,-0.06,1.94,-5.92,14.28,-39.95,0.13,-0.34,...,1,1,0,0,1,0,0,1,0,0
115384,-8.21,"136,512.62",0.08,-0.06,1.94,-5.92,14.28,-39.95,0.13,-0.34,...,1,0,1,0,0,1,0,0,1,0
115385,-5.00,"194,676.00",0.08,-0.06,0.00,0.00,14.28,-39.95,0.13,-0.34,...,0,0,1,0,0,1,0,0,1,0


## Red neuronal 

In [17]:
#Normalizo data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X_train)

In [54]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(100, activation='tanh', input_dim=593))
model.add(Dense(100, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

In [55]:
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer= opt, metrics=[keras.metrics.AUC(
    num_thresholds=20,
    curve="ROC",
    summation_method="interpolation",
    name=None,
    dtype=None,
    thresholds=None,
    multi_label=False,
    num_labels=None,
    label_weights=None,
    from_logits=False,
)
])

In [56]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 100)               59400     
                                                                 
 dense_20 (Dense)            (None, 100)               10100     
                                                                 
 dense_21 (Dense)            (None, 1)                 101       
                                                                 
Total params: 69,601
Trainable params: 69,601
Non-trainable params: 0
_________________________________________________________________


In [57]:
history = model.fit(X, y_train.values , epochs=50, batch_size = 1024)

Epoch 1/50
113/113 [==============================] - 2s 10ms/step - loss: 0.1589 - auc_4: 0.7423
Epoch 2/50
113/113 [==============================] - 1s 9ms/step - loss: 0.0713 - auc_4: 0.8138
Epoch 3/50
113/113 [==============================] - 1s 9ms/step - loss: 0.0628 - auc_4: 0.8391
Epoch 4/50
113/113 [==============================] - 1s 9ms/step - loss: 0.0577 - auc_4: 0.8514
Epoch 5/50
113/113 [==============================] - 1s 9ms/step - loss: 0.0535 - auc_4: 0.8632
Epoch 6/50
113/113 [==============================] - 1s 9ms/step - loss: 0.0499 - auc_4: 0.8789
Epoch 7/50
113/113 [==============================] - 1s 9ms/step - loss: 0.0464 - auc_4: 0.8903
Epoch 8/50
113/113 [==============================] - 1s 9ms/step - loss: 0.0440 - auc_4: 0.8940
Epoch 9/50
113/113 [==============================] - 1s 10ms/step - loss: 0.0410 - auc_4: 0.9036
Epoch 10/50
113/113 [==============================] - 1s 11ms/step - loss: 0.0376 - auc_4: 0.9127
Epoch 11/50
113/113 [=====

In [62]:
scaledTest = sc.transform(X_test)
preds = model.predict(scaledTest)

In [63]:
roc_auc_score(np.asarray(y_test).flatten(), preds)

0.7625183295904044

## Comparar features XGBoost y Logistic Regression

Importancias XGBoost<img src="xgboostimport.png" />

Importancias Logistic Regresion <img src="regresimport.png" />

Parece como que toman diferentes importancias en los features entre ambos modelos